<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/TF_kogpt2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer, GPT2Config
from transformers import TFGPT2LMHeadModel
from tensorflow.keras.optimizers.schedules import LearningRateSchedule

모델 호출

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

special_tokens = {'bos_token': '</s>', 'eos_token': '</s>', 'pad_token': '<pad>', 'unk_token' : '<unk>'}
tokenizer.add_special_tokens(special_tokens)

model_config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')

model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', config=model_config, from_pt = True)

model.resize_token_embeddings(len(tokenizer))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

데이터셋 구축

In [5]:
import pandas as pd
import tqdm

In [6]:

# Initialize empty lists to store data
sentence1_list = []
sentence3_list = []
output_list = []

# Read the JSONL file line by line and extract data
with open("/content/drive/MyDrive/nikluge-sc-2023-train.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        output = data['output']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)
        output_list.append(output)

with open("/content/drive/MyDrive/nikluge-sc-2023-dev.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        output = data['output']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)
        output_list.append(output)

# Create a Pandas DataFrame
df = pd.DataFrame({'sentence1': sentence1_list, 'sentence3': sentence3_list, 'output': output_list})

In [7]:
train_data = df

In [8]:
batch_size = 16

In [9]:
def get_story_data():
  for sentence1, sentence3, output in zip(train_data.sentence1.to_list(), train_data.sentence3.to_list(), train_data.output.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<sys>' + sentence1 +'<unused0>' + sentence3 + '<usr>' + output)
    yield bos_token + sent + eos_token

In [10]:
train_dataset = tf.data.Dataset.from_generator(get_story_data, output_types=tf.int32)
train_dataset = train_dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [11]:
adam = tf.keras.optimizers.Adam(learning_rate=5e-6, epsilon=1e-08)
steps = len(train_data) // batch_size + 1
print(steps)

8448


In [12]:
class CustomLearningRateSchedule(LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, decay_rate):
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate

    def __call__(self, step):
        return self.initial_learning_rate * (self.decay_rate ** (step / self.decay_steps))

In [13]:
initial_learning_rate = 1e-4
decay_steps = steps // 4  # You can adjust this value as needed
decay_rate = 0.5  # You can adjust this value as needed

learning_rate_schedule = CustomLearningRateSchedule(initial_learning_rate, decay_steps, decay_rate)
adam = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule, epsilon=1e-08)


In [14]:
@tf.function(reduce_retracing=True)
def training_step(batch):
    with tf.GradientTape() as tape:
        result = model(batch, labels=batch)
        loss = result[0]
        batch_loss = tf.reduce_mean(loss)

    grads = tape.gradient(batch_loss, model.trainable_variables)
    adam.apply_gradients(zip(grads, model.trainable_variables))
    return batch_loss


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    epoch_loss = 0

    for step, batch in enumerate(tqdm.tqdm_notebook(train_dataset, total=steps)):
        batch_loss = training_step(batch)
        epoch_loss += batch_loss / steps

        # Update the learning rate at each step
        learning_rate = learning_rate_schedule(step)
        adam.learning_rate.assign(learning_rate)

    print('[Epoch: {:>6}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

    output_dir = "Tiabet-TFkoGPT-complete_story-epoch-{}".format(epoch+1)
    model.push_to_hub(output_dir)


<ipython-input-15-ac1e7b5f31b7>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm.tqdm_notebook(train_dataset, total=steps)):


  0%|          | 0/8448 [00:00<?, ?it/s]

[Epoch:      1] cost = 2.10969758


tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]

<ipython-input-15-ac1e7b5f31b7>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm.tqdm_notebook(train_dataset, total=steps)):


  0%|          | 0/8448 [00:00<?, ?it/s]

[Epoch:      2] cost = 1.73808086


tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]

<ipython-input-15-ac1e7b5f31b7>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in enumerate(tqdm.tqdm_notebook(train_dataset, total=steps)):


  0%|          | 0/8448 [00:00<?, ?it/s]